In [1]:
import pandas as pd
import numpy as np
import re
import traceback
import sys
from variables import * 
import tqdm
import json

In [2]:
df = pd.read_csv('dataset/Influenza_Vaccination_Coverage.csv')
df.head()

,Vaccine,Geography Type,Geography,FIPS,Season/Survey Year,Month,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
0,Seasonal Influenza,Counties,New Haven,9009,2018,1,>=18 Years,Non-Medical Setting,45.5,43.9 to 47.2,NaN
1,Seasonal Influenza,Counties,New Haven,9009,2021,1,>=18 Years,Non-Medical Setting,53.0,46.0 to 60.9,NaN
2,Seasonal Influenza,Counties,New Haven,9009,2020,1,Age,>=18 Years,52.4,50.6 to 54.3,NaN
3,Seasonal Influenza,Counties,New Haven,9009,2021,1,Age,>=18 Years,50.2,45.4 to 55.8,NaN
4,Seasonal Influenza,Counties,New Haven,9009,2018,1,Age,>=18 Years,34.0,32.6 to 35.5,NaN


In [3]:
df['Vaccine'].unique()

array(['Seasonal Influenza',
       'Any Influenza Vaccination, Seasonal or H1N1',
       'Influenza A (H1N1) 2009 Monovalent'], dtype=object)

In [4]:
df.shape[0]

220729

In [5]:
# clean the vaccines first
df['Vaccine'] = df['Vaccine'].replace('Seasonal Influenza', 'Influenza')
df.head()

,Vaccine,Geography Type,Geography,FIPS,Season/Survey Year,Month,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
0,Influenza,Counties,New Haven,9009,2018,1,>=18 Years,Non-Medical Setting,45.5,43.9 to 47.2,NaN
1,Influenza,Counties,New Haven,9009,2021,1,>=18 Years,Non-Medical Setting,53.0,46.0 to 60.9,NaN
2,Influenza,Counties,New Haven,9009,2020,1,Age,>=18 Years,52.4,50.6 to 54.3,NaN
3,Influenza,Counties,New Haven,9009,2021,1,Age,>=18 Years,50.2,45.4 to 55.8,NaN
4,Influenza,Counties,New Haven,9009,2018,1,Age,>=18 Years,34.0,32.6 to 35.5,NaN


In [6]:
df['Sample Size'] = pd.to_numeric(df['Sample Size'], errors='coerce').astype('Int64')
df.head()

,Vaccine,Geography Type,Geography,FIPS,Season/Survey Year,Month,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
0,Influenza,Counties,New Haven,9009,2018,1,>=18 Years,Non-Medical Setting,45.5,43.9 to 47.2,<NA>
1,Influenza,Counties,New Haven,9009,2021,1,>=18 Years,Non-Medical Setting,53.0,46.0 to 60.9,<NA>
2,Influenza,Counties,New Haven,9009,2020,1,Age,>=18 Years,52.4,50.6 to 54.3,<NA>
3,Influenza,Counties,New Haven,9009,2021,1,Age,>=18 Years,50.2,45.4 to 55.8,<NA>
4,Influenza,Counties,New Haven,9009,2018,1,Age,>=18 Years,34.0,32.6 to 35.5,<NA>


In [7]:
df[df['95% CI (%)'].astype(str).str.contains(r'[^0-9.\s%to]', na=False)]

,Vaccine,Geography Type,Geography,FIPS,Season/Survey Year,Month,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
6,Influenza,States/Local Areas,New Jersey,34,2009-10,5,Race and Ethnicity,"American Indian or Alaska Native, Non-Hispanic",57.7,40.2 to 75.2 ‡,48
7,Influenza,States/Local Areas,New Jersey,34,2009-10,4,Race and Ethnicity,"American Indian or Alaska Native, Non-Hispanic",57.7,40.2 to 75.2 ‡,48
11,Influenza,States/Local Areas,New Jersey,34,2009-10,8,Race and Ethnicity,"Asian, Non-Hispanic",NR †,NR †,552
12,Influenza,States/Local Areas,New Jersey,34,2009-10,9,Race and Ethnicity,"American Indian or Alaska Native, Non-Hispanic",NR †,NR †,48
13,Influenza,States/Local Areas,New Jersey,34,2009-10,1,Race and Ethnicity,"American Indian or Alaska Native, Non-Hispanic",57.7,40.2 to 75.2 ‡,48
...,...,...,...,...,...,...,...,...,...,...,...
220712,Influenza,States/Local Areas,Maine,23,2009-10,1,Race and Ethnicity,"Asian, Non-Hispanic",70.3,58.8 to 81.8 ‡,49
220713,Influenza,States/Local Areas,Maine,23,2009-10,10,Race and Ethnicity,"Black, Non-Hispanic",NR †,NR †,50
220714,Influenza,States/Local Areas,Maine,23,2009-10,11,Race and Ethnicity,"Black, Non-Hispanic",30.7,13.8 to 47.6 ‡,50
220715,Influenza,States/Local Areas,Maine,23,2009-10,12,Race and Ethnicity,"Black, Non-Hispanic",30.7,13.8 to 47.6 ‡,50


In [8]:
ci = df['95% CI (%)'].astype(str).str.replace('%', '', regex=False).str.strip()
df[['ci_lower', 'ci_upper']] = ci.str.extract(r'([\d.]+)\s+to\s+([\d.]+)')
df[['ci_lower', 'ci_upper']] = ci.str.split(r'\s+to\s+', expand=True)
df.head()

,Vaccine,Geography Type,Geography,FIPS,Season/Survey Year,Month,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,ci_lower,ci_upper
0,Influenza,Counties,New Haven,9009,2018,1,>=18 Years,Non-Medical Setting,45.5,43.9 to 47.2,<NA>,43.9,47.2
1,Influenza,Counties,New Haven,9009,2021,1,>=18 Years,Non-Medical Setting,53.0,46.0 to 60.9,<NA>,46.0,60.9
2,Influenza,Counties,New Haven,9009,2020,1,Age,>=18 Years,52.4,50.6 to 54.3,<NA>,50.6,54.3
3,Influenza,Counties,New Haven,9009,2021,1,Age,>=18 Years,50.2,45.4 to 55.8,<NA>,45.4,55.8
4,Influenza,Counties,New Haven,9009,2018,1,Age,>=18 Years,34.0,32.6 to 35.5,<NA>,32.6,35.5


In [9]:
# getting the vaccine dimension table from postgres
import psycopg2 as pg
from creds import POSTGRES_USERNAME, POSTGRES_PW
conn = pg.connect(
    dbname='cs689_project',  
    user=POSTGRES_USERNAME,
    password=POSTGRES_PW,
    host='localhost',
    port='5432'
)
cursor = conn.cursor()

In [10]:
cursor.execute("SELECT vaccine_id, cleaned_vaccine, start_date, vaccine_strains FROM vaccine_dim")
mapping_data = cursor.fetchall()

vaccine_mapping = pd.DataFrame(mapping_data, columns=["vaccine_id", "cleaned_vaccine", "start_date", "vaccine_strains"])
vaccine_mapping

,vaccine_id,cleaned_vaccine,start_date,vaccine_strains
0,1,DTaP,2011-09-01,"Diphtheria, Tetanus, and Bordetella pertussis ..."
1,2,Polio,2011-09-01,"Poliovirus Types 1, 2, and 3 (inactivated)"
2,3,Hep B,2011-09-01,Hepatitis B virus (HBV)
3,4,PCV,2011-09-01,Streptococcus pneumoniae — multiple serotypes ...
4,7,Hib,2011-09-01,Haemophilus influenzae type b
5,10,Combined 7 Series,2011-09-01,"Includes: DTaP, IPV, MMR, Hib, Hep B, Varicell..."
6,11,Rotavirus,2011-09-01,"Rotavirus types G1, G2, G3, G4, G9 (Rotarix); ..."
7,9,Influenza,2011-09-01,"A/California/7/2009, A/Victoria/361/2011 (H3N2..."
8,12,Influenza,2012-09-01,"A/California/7/2009, A/Victoria/361/2011 (H3N2..."
9,13,Influenza,2013-09-01,"A/California/7/2009, A/Texas/50/2012 (H3N2), B..."


In [11]:
vaccine_mapping['start_date'] = pd.to_datetime(vaccine_mapping['start_date'])
vaccine_mapping['year'] = vaccine_mapping['start_date'].dt.year
vaccine_mapping

,vaccine_id,cleaned_vaccine,start_date,vaccine_strains,year
0,1,DTaP,2011-09-01,"Diphtheria, Tetanus, and Bordetella pertussis ...",2011
1,2,Polio,2011-09-01,"Poliovirus Types 1, 2, and 3 (inactivated)",2011
2,3,Hep B,2011-09-01,Hepatitis B virus (HBV),2011
3,4,PCV,2011-09-01,Streptococcus pneumoniae — multiple serotypes ...,2011
4,7,Hib,2011-09-01,Haemophilus influenzae type b,2011
5,10,Combined 7 Series,2011-09-01,"Includes: DTaP, IPV, MMR, Hib, Hep B, Varicell...",2011
6,11,Rotavirus,2011-09-01,"Rotavirus types G1, G2, G3, G4, G9 (Rotarix); ...",2011
7,9,Influenza,2011-09-01,"A/California/7/2009, A/Victoria/361/2011 (H3N2...",2011
8,12,Influenza,2012-09-01,"A/California/7/2009, A/Victoria/361/2011 (H3N2...",2012
9,13,Influenza,2013-09-01,"A/California/7/2009, A/Texas/50/2012 (H3N2), B...",2013


In [12]:
df_counties = df[df['Geography Type'] == 'Counties']
df_counties.head()

,Vaccine,Geography Type,Geography,FIPS,Season/Survey Year,Month,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,ci_lower,ci_upper
0,Influenza,Counties,New Haven,9009,2018,1,>=18 Years,Non-Medical Setting,45.5,43.9 to 47.2,<NA>,43.9,47.2
1,Influenza,Counties,New Haven,9009,2021,1,>=18 Years,Non-Medical Setting,53.0,46.0 to 60.9,<NA>,46.0,60.9
2,Influenza,Counties,New Haven,9009,2020,1,Age,>=18 Years,52.4,50.6 to 54.3,<NA>,50.6,54.3
3,Influenza,Counties,New Haven,9009,2021,1,Age,>=18 Years,50.2,45.4 to 55.8,<NA>,45.4,55.8
4,Influenza,Counties,New Haven,9009,2018,1,Age,>=18 Years,34.0,32.6 to 35.5,<NA>,32.6,35.5


In [13]:
original_length = df_counties.shape[0]
original_length

21818

In [14]:
for _, row in df_counties.iterrows():
    start_year = row['Season/Survey Year'][:4]
    df_counties.loc[_, 'start_cohort_year'] = start_year
    df_counties.loc[_, 'start_cohort_datetime'] = str(start_year) + '-' + str(row['Month'])
df_counties.head()

/var/folders/jq/q12_1s5j0kx30cqlm76w1m740000gn/T/ipykernel_12858/294772658.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_counties.loc[_, 'start_cohort_year'] = start_year
/var/folders/jq/q12_1s5j0kx30cqlm76w1m740000gn/T/ipykernel_12858/294772658.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_counties.loc[_, 'start_cohort_datetime'] = str(start_year) + '-' + str(row['Month'])


,Vaccine,Geography Type,Geography,FIPS,Season/Survey Year,Month,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,ci_lower,ci_upper,start_cohort_year,start_cohort_datetime
0,Influenza,Counties,New Haven,9009,2018,1,>=18 Years,Non-Medical Setting,45.5,43.9 to 47.2,<NA>,43.9,47.2,2018,2018-1
1,Influenza,Counties,New Haven,9009,2021,1,>=18 Years,Non-Medical Setting,53.0,46.0 to 60.9,<NA>,46.0,60.9,2021,2021-1
2,Influenza,Counties,New Haven,9009,2020,1,Age,>=18 Years,52.4,50.6 to 54.3,<NA>,50.6,54.3,2020,2020-1
3,Influenza,Counties,New Haven,9009,2021,1,Age,>=18 Years,50.2,45.4 to 55.8,<NA>,45.4,55.8,2021,2021-1
4,Influenza,Counties,New Haven,9009,2018,1,Age,>=18 Years,34.0,32.6 to 35.5,<NA>,32.6,35.5,2018,2018-1


In [15]:
df_counties['start_cohort_year'] = (pd.to_numeric(df_counties['start_cohort_year'], errors='coerce').astype('Int64'))
df_counties = pd.merge(df_counties, vaccine_mapping, left_on = ['Vaccine', 'start_cohort_year'], right_on=['cleaned_vaccine', 'year'], how='left')
df_counties.head()

/var/folders/jq/q12_1s5j0kx30cqlm76w1m740000gn/T/ipykernel_12858/3523382953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_counties['start_cohort_year'] = (pd.to_numeric(df_counties['start_cohort_year'], errors='coerce').astype('Int64'))


,Vaccine,Geography Type,Geography,FIPS,Season/Survey Year,Month,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,ci_lower,ci_upper,start_cohort_year,start_cohort_datetime,vaccine_id,cleaned_vaccine,start_date,vaccine_strains,year
0,Influenza,Counties,New Haven,9009,2018,1,>=18 Years,Non-Medical Setting,45.5,43.9 to 47.2,<NA>,43.9,47.2,2018,2018-1,18,Influenza,2018-09-01,"A/Michigan/45/2015, A/Singapore/INFIMH-16-0019...",2018
1,Influenza,Counties,New Haven,9009,2021,1,>=18 Years,Non-Medical Setting,53.0,46.0 to 60.9,<NA>,46.0,60.9,2021,2021-1,21,Influenza,2021-09-01,"A/Victoria/2570/2019, A/Cambodia/e0826360/2020...",2021
2,Influenza,Counties,New Haven,9009,2020,1,Age,>=18 Years,52.4,50.6 to 54.3,<NA>,50.6,54.3,2020,2020-1,20,Influenza,2020-09-01,"A/Guangdong-Maonan/SWL1536/2019, A/Hong Kong/2...",2020
3,Influenza,Counties,New Haven,9009,2021,1,Age,>=18 Years,50.2,45.4 to 55.8,<NA>,45.4,55.8,2021,2021-1,21,Influenza,2021-09-01,"A/Victoria/2570/2019, A/Cambodia/e0826360/2020...",2021
4,Influenza,Counties,New Haven,9009,2018,1,Age,>=18 Years,34.0,32.6 to 35.5,<NA>,32.6,35.5,2018,2018-1,18,Influenza,2018-09-01,"A/Michigan/45/2015, A/Singapore/INFIMH-16-0019...",2018


In [16]:
df_counties[df_counties['vaccine_id'].isna()]

,Vaccine,Geography Type,Geography,FIPS,Season/Survey Year,Month,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,ci_lower,ci_upper,start_cohort_year,start_cohort_datetime,vaccine_id,cleaned_vaccine,start_date,vaccine_strains,year


In [17]:
df_counties.drop(columns=['Vaccine', 'cleaned_vaccine','vaccine_strains', 'year', 'start_date', 'Season/Survey Year', 'start_cohort_year', 'Month', '95% CI (%)'], inplace=True)
df_counties.head()

,Geography Type,Geography,FIPS,Dimension Type,Dimension,Estimate (%),Sample Size,ci_lower,ci_upper,start_cohort_datetime,vaccine_id
0,Counties,New Haven,9009,>=18 Years,Non-Medical Setting,45.5,<NA>,43.9,47.2,2018-1,18
1,Counties,New Haven,9009,>=18 Years,Non-Medical Setting,53.0,<NA>,46.0,60.9,2021-1,21
2,Counties,New Haven,9009,Age,>=18 Years,52.4,<NA>,50.6,54.3,2020-1,20
3,Counties,New Haven,9009,Age,>=18 Years,50.2,<NA>,45.4,55.8,2021-1,21
4,Counties,New Haven,9009,Age,>=18 Years,34.0,<NA>,32.6,35.5,2018-1,18


In [18]:
df_counties['start_cohort_datetime'] = pd.to_datetime(df_counties['start_cohort_datetime']).dt.strftime('%Y-%m-%d')
df_counties.head()

,Geography Type,Geography,FIPS,Dimension Type,Dimension,Estimate (%),Sample Size,ci_lower,ci_upper,start_cohort_datetime,vaccine_id
0,Counties,New Haven,9009,>=18 Years,Non-Medical Setting,45.5,<NA>,43.9,47.2,2018-01-01,18
1,Counties,New Haven,9009,>=18 Years,Non-Medical Setting,53.0,<NA>,46.0,60.9,2021-01-01,21
2,Counties,New Haven,9009,Age,>=18 Years,52.4,<NA>,50.6,54.3,2020-01-01,20
3,Counties,New Haven,9009,Age,>=18 Years,50.2,<NA>,45.4,55.8,2021-01-01,21
4,Counties,New Haven,9009,Age,>=18 Years,34.0,<NA>,32.6,35.5,2018-01-01,18


In [19]:
# getting state from FIPS
df_counties['FIPS_clean'] = df_counties['FIPS'].astype(str)
for _,row in df_counties.iterrows():
    if len(row['FIPS_clean']) == 4:
        # print(row['FIPS_clean'])
        df_counties.loc[_,'FIPS_clean'] = '0'+ row['FIPS_clean']
df_counties.head()

,Geography Type,Geography,FIPS,Dimension Type,Dimension,Estimate (%),Sample Size,ci_lower,ci_upper,start_cohort_datetime,vaccine_id,FIPS_clean
0,Counties,New Haven,9009,>=18 Years,Non-Medical Setting,45.5,<NA>,43.9,47.2,2018-01-01,18,09009
1,Counties,New Haven,9009,>=18 Years,Non-Medical Setting,53.0,<NA>,46.0,60.9,2021-01-01,21,09009
2,Counties,New Haven,9009,Age,>=18 Years,52.4,<NA>,50.6,54.3,2020-01-01,20,09009
3,Counties,New Haven,9009,Age,>=18 Years,50.2,<NA>,45.4,55.8,2021-01-01,21,09009
4,Counties,New Haven,9009,Age,>=18 Years,34.0,<NA>,32.6,35.5,2018-01-01,18,09009


In [95]:
# getting the state from counties using FIPS
# im not going to call the API 200k+ times, so im going to store the retrieved FIPS information in a dictionary
import requests
from tqdm import tqdm
import time
county_state = {}
chunk_size = 10

# base URL for the Census Bureau 2020 Decennial Census Public Law 94-171 Redistricting Data
url = "https://api.census.gov/data/2020/dec/pl"
## key = key

# Set up the parameters for the API call:
def chunk_dataframe(df, chunk_size):
    for i in range(0, len(df), chunk_size):
        yield df.iloc[i:i+chunk_size]

for chunk in tqdm(chunk_dataframe(df_counties, chunk_size), desc="Processing chunks"):
    for _, row in tqdm(chunk.iterrows(), total=len(chunk), leave=False, desc="Fetching FIPS"):
        fips = row['FIPS_clean']
        if fips in county_state:
            continue
        
        state = fips[:2]
        county = fips[2:]

        params = {
            "get": "NAME",
            "for": f"county:{county}",
            "in": f"state:{state}",
            "key": key

        }

        try:
            response = requests.get(url, params=params, timeout=10)

            if response.status_code == 200:
                data = response.json()
                county_info = data[1]
                county_state[fips] = county_info[0]
            else:
                print(f"Error for {fips}: {response.status_code} - {response.text}")


        except requests.exceptions.Timeout:
            print(f"Timeout for {fips}")
        except Exception as e:
            print(f"Request error for {fips}:", e)
    
    # optional sleep between chunks
    time.sleep(1)

Processing chunks: 0it [00:02, ?it/s]


KeyboardInterrupt: 

In [168]:
len(county_state)

3143

In [169]:
county_state

{'09009': 'New Haven County, Connecticut',
 '18035': 'Delaware County, Indiana',
 '19055': 'Delaware County, Iowa',
 '36025': 'Delaware County, New York',
 '39041': 'Delaware County, Ohio',
 '40041': 'Delaware County, Oklahoma',
 '42045': 'Delaware County, Pennsylvania',
 '08029': 'Delta County, Colorado',
 '26041': 'Delta County, Michigan',
 '48119': 'Delta County, Texas',
 '02068': 'Denali Borough, Alaska',
 '29065': 'Dent County, Missouri',
 '48121': 'Denton County, Texas',
 '08031': 'Denver County, Colorado',
 '19057': 'Des Moines County, Iowa',
 '41017': 'Deschutes County, Oregon',
 '05041': 'Desha County, Arkansas',
 '12027': 'DeSoto County, Florida',
 '28033': 'DeSoto County, Mississippi',
 '31049': 'Deuel County, Nebraska',
 '46039': 'Deuel County, South Dakota',
 '40043': 'Dewey County, Oklahoma',
 '46041': 'Dewey County, South Dakota',
 '48123': 'DeWitt County, Texas',
 '48125': 'Dickens County, Texas',
 '51051': 'Dickenson County, Virginia',
 '38021': 'Dickey County, North D

## DANGER ZONE! 
### Running the script below without running the script above will overwrite the json file!

In [170]:
with open('fips_mapping.json', 'w') as f:
    json.dump(county_state, f)

In [20]:
import json
with open('fips_mapping.json', 'r') as file:
    data = json.load(file)
data

{'09009': 'New Haven County, Connecticut',
 '18035': 'Delaware County, Indiana',
 '19055': 'Delaware County, Iowa',
 '36025': 'Delaware County, New York',
 '39041': 'Delaware County, Ohio',
 '40041': 'Delaware County, Oklahoma',
 '42045': 'Delaware County, Pennsylvania',
 '08029': 'Delta County, Colorado',
 '26041': 'Delta County, Michigan',
 '48119': 'Delta County, Texas',
 '02068': 'Denali Borough, Alaska',
 '29065': 'Dent County, Missouri',
 '48121': 'Denton County, Texas',
 '08031': 'Denver County, Colorado',
 '19057': 'Des Moines County, Iowa',
 '41017': 'Deschutes County, Oregon',
 '05041': 'Desha County, Arkansas',
 '12027': 'DeSoto County, Florida',
 '28033': 'DeSoto County, Mississippi',
 '31049': 'Deuel County, Nebraska',
 '46039': 'Deuel County, South Dakota',
 '40043': 'Dewey County, Oklahoma',
 '46041': 'Dewey County, South Dakota',
 '48123': 'DeWitt County, Texas',
 '48125': 'Dickens County, Texas',
 '51051': 'Dickenson County, Virginia',
 '38021': 'Dickey County, North D

In [21]:
data['09009']

'New Haven County, Connecticut'

In [22]:
df_county_list = df = pd.DataFrame([
    {'FIPS': k, 'County': v.rsplit(',', 1)[0], 'State': v.rsplit(',', 1)[1].strip()}
    for k, v in data.items()
])

In [23]:
df_county_list.shape[0]

3143

In [26]:
# filling in counties missed by API
missing_locations = [
    {'FIPS': '02261', 'County': 'Valdez-Cordova Census Area', 'State': 'Alaska'},
    {'FIPS': '09110', 'County': 'Capitol Planning Region', 'State': 'Connecticut'},
    {'FIPS': '09120', 'County': 'Greater Bridgeport Planning Region', 'State': 'Connecticut'},
    {'FIPS': '09130', 'County': 'Greater Bridgeport Planning Region', 'State': 'Connecticut'},
    {'FIPS': '09140', 'County': 'Greater Bridgeport Planning Region', 'State': 'Connecticut'},
    {'FIPS': '09150', 'County': 'Northeastern Connecticut Planning Region', 'State': 'Connecticut'},
    {'FIPS': '09160', 'County': 'Northwest Hills Planning Region', 'State': 'Connecticut'},
    {'FIPS': '09170', 'County': 'South Central Connecticut Planning Region', 'State': 'Connecticut'},
    {'FIPS': '09180', 'County': 'Southeastern Connecticut Planning Region', 'State': 'Connecticut'},
    {'FIPS': '09190', 'County': 'Western Connecticut Planning Region', 'State': 'Connecticut'},
    {'FIPS': '21179', 'County': 'Nelson County', 'State': 'Kentucky'},
    {'FIPS': '48219', 'County': 'Hockley County', 'State': 'Texas'}
]

for row in missing_locations:
    df_county_list.loc[len(df_county_list)] = row

In [27]:
df_county_list[df_county_list['FIPS'] == '02261']

,FIPS,County,State
3143,02261,Valdez-Cordova Census Area,Alaska


In [28]:
for idx, row in df_county_list.iterrows():
    state = row['State']
    region = state_to_region.get(state)
    df_county_list.loc[idx, 'Region'] = region
df_county_list.tail()

,FIPS,County,State,Region
3150,09170,South Central Connecticut Planning Region,Connecticut,1.0
3151,09180,Southeastern Connecticut Planning Region,Connecticut,1.0
3152,09190,Western Connecticut Planning Region,Connecticut,1.0
3153,21179,Nelson County,Kentucky,4.0
3154,48219,Hockley County,Texas,6.0


In [29]:
df_county_list['Region'] = pd.to_numeric(df_county_list['Region']).astype('Int64')
df_county_list.head()

,FIPS,County,State,Region
0,09009,New Haven County,Connecticut,1
1,18035,Delaware County,Indiana,5
2,19055,Delaware County,Iowa,7
3,36025,Delaware County,New York,2
4,39041,Delaware County,Ohio,5


In [30]:
df_counties.head()

,Geography Type,Geography,FIPS,Dimension Type,Dimension,Estimate (%),Sample Size,ci_lower,ci_upper,start_cohort_datetime,vaccine_id,FIPS_clean
0,Counties,New Haven,9009,>=18 Years,Non-Medical Setting,45.5,<NA>,43.9,47.2,2018-01-01,18,09009
1,Counties,New Haven,9009,>=18 Years,Non-Medical Setting,53.0,<NA>,46.0,60.9,2021-01-01,21,09009
2,Counties,New Haven,9009,Age,>=18 Years,52.4,<NA>,50.6,54.3,2020-01-01,20,09009
3,Counties,New Haven,9009,Age,>=18 Years,50.2,<NA>,45.4,55.8,2021-01-01,21,09009
4,Counties,New Haven,9009,Age,>=18 Years,34.0,<NA>,32.6,35.5,2018-01-01,18,09009


In [31]:
df_counties = pd.merge(df_counties, df_county_list, left_on=['FIPS_clean'], right_on = ['FIPS'], how = 'left')
df_counties.head()

,Geography Type,Geography,FIPS_x,Dimension Type,Dimension,Estimate (%),Sample Size,ci_lower,ci_upper,start_cohort_datetime,vaccine_id,FIPS_clean,FIPS_y,County,State,Region
0,Counties,New Haven,9009,>=18 Years,Non-Medical Setting,45.5,<NA>,43.9,47.2,2018-01-01,18,09009,09009,New Haven County,Connecticut,1
1,Counties,New Haven,9009,>=18 Years,Non-Medical Setting,53.0,<NA>,46.0,60.9,2021-01-01,21,09009,09009,New Haven County,Connecticut,1
2,Counties,New Haven,9009,Age,>=18 Years,52.4,<NA>,50.6,54.3,2020-01-01,20,09009,09009,New Haven County,Connecticut,1
3,Counties,New Haven,9009,Age,>=18 Years,50.2,<NA>,45.4,55.8,2021-01-01,21,09009,09009,New Haven County,Connecticut,1
4,Counties,New Haven,9009,Age,>=18 Years,34.0,<NA>,32.6,35.5,2018-01-01,18,09009,09009,New Haven County,Connecticut,1


In [32]:
df_counties.shape[0]

21832

In [33]:
df_county_list.shape[0]

3155

In [34]:
df_county_list = df_county_list.drop_duplicates(subset='FIPS')


In [38]:
df_counties_unique = df_counties.drop_duplicates(subset='FIPS_clean')

df_county_list = pd.merge(
    df_county_list,
    df_counties_unique,
    left_on='FIPS',
    right_on='FIPS_clean',
    how='left'
)

In [39]:
df_county_list.head()

,FIPS,County,State,Region
0,09009,New Haven County,Connecticut,1
1,18035,Delaware County,Indiana,5
2,19055,Delaware County,Iowa,7
3,36025,Delaware County,New York,2
4,39041,Delaware County,Ohio,5


In [40]:
df_county.shape[0]

3153

In [222]:
try:
    insert_query = '''
    INSERT INTO fips_dim (
        state_territory, 
        county, 
        region, 
        original_geography_type, 
        original_geography,
        fips
    )
    VALUES (%s, %s, %s, %s, %s, %s)
    ON CONFLICT (original_geography, state_territory, county) DO NOTHING

    '''
    for index, row in df_county.iterrows():
        values = (
        row['State_y'] if pd.notna(row['State_y']) else None,
        row['County_y'] if pd.notna(row['County_y']) else None,
        row['Region_y'] if pd.notna(row['Region_y']) else None,
        row['Geography Type'] if pd.notna(row['Geography Type']) else None,
        row['Geography'] if pd.notna(row['Geography']) else None, 
        row['FIPS_clean'] if pd.notna(row['FIPS_clean']) else None
    )

        cursor.execute(insert_query,values)
    conn.commit()
    print(f"{len(df_county)} records inserted into geography_dim.")

except Exception as e:
    print(f"Error occurred: {e}")
    conn.rollback()
    

3153 records inserted into geography_dim.


In [41]:
df_counties.head()

,Geography Type,Geography,FIPS_x,Dimension Type,Dimension,Estimate (%),Sample Size,ci_lower,ci_upper,start_cohort_datetime,vaccine_id,FIPS_clean,FIPS_y,County,State,Region
0,Counties,New Haven,9009,>=18 Years,Non-Medical Setting,45.5,<NA>,43.9,47.2,2018-01-01,18,09009,09009,New Haven County,Connecticut,1
1,Counties,New Haven,9009,>=18 Years,Non-Medical Setting,53.0,<NA>,46.0,60.9,2021-01-01,21,09009,09009,New Haven County,Connecticut,1
2,Counties,New Haven,9009,Age,>=18 Years,52.4,<NA>,50.6,54.3,2020-01-01,20,09009,09009,New Haven County,Connecticut,1
3,Counties,New Haven,9009,Age,>=18 Years,50.2,<NA>,45.4,55.8,2021-01-01,21,09009,09009,New Haven County,Connecticut,1
4,Counties,New Haven,9009,Age,>=18 Years,34.0,<NA>,32.6,35.5,2018-01-01,18,09009,09009,New Haven County,Connecticut,1


In [42]:
df_counties.drop(columns=['FIPS_x', 'FIPS_y'], inplace=True)
df_counties.head()

,Geography Type,Geography,Dimension Type,Dimension,Estimate (%),Sample Size,ci_lower,ci_upper,start_cohort_datetime,vaccine_id,FIPS_clean,County,State,Region
0,Counties,New Haven,>=18 Years,Non-Medical Setting,45.5,<NA>,43.9,47.2,2018-01-01,18,09009,New Haven County,Connecticut,1
1,Counties,New Haven,>=18 Years,Non-Medical Setting,53.0,<NA>,46.0,60.9,2021-01-01,21,09009,New Haven County,Connecticut,1
2,Counties,New Haven,Age,>=18 Years,52.4,<NA>,50.6,54.3,2020-01-01,20,09009,New Haven County,Connecticut,1
3,Counties,New Haven,Age,>=18 Years,50.2,<NA>,45.4,55.8,2021-01-01,21,09009,New Haven County,Connecticut,1
4,Counties,New Haven,Age,>=18 Years,34.0,<NA>,32.6,35.5,2018-01-01,18,09009,New Haven County,Connecticut,1


In [43]:
cursor.execute("SELECT fips_id, original_geography, original_geography_type, fips FROM fips_dim")
mapping_data = cursor.fetchall()

fips_mapping = pd.DataFrame(mapping_data, columns=["fips_id", "original_geography", "original_geography_type", "fips"])
fips_mapping
fips_mapping['fips'] = pd.to_numeric(fips_mapping['fips']).astype('Int64')

In [44]:
fips_mapping['fips'] = pd.to_numeric(fips_mapping['fips']).astype('Int64')
fips_mapping

,fips_id,original_geography,original_geography_type,fips
0,1,New Haven,Counties,9009
1,2,Delaware,Counties,18035
2,8,Delta,Counties,8029
3,11,Denali Borough,Counties,2068
4,12,Dent,Counties,29065
...,...,...,...,...
3157,12587,Northeastern Connecticut,Counties,9150
3158,12588,Northwest Hills,Counties,9160
3159,12589,South Central Connecticut,Counties,9170
3160,12590,Southeastern Connecticut,Counties,9180


In [45]:
df_counties['FIPS_clean'] = df_counties['FIPS_clean'].astype('Int64')
df_counties = pd.merge(df_counties, fips_mapping, left_on=['FIPS_clean'], right_on = ['fips'], how = 'left')
print(df_counties.shape[0])
df_counties.head()

21832


,Geography Type,Geography,Dimension Type,Dimension,Estimate (%),Sample Size,ci_lower,ci_upper,start_cohort_datetime,vaccine_id,FIPS_clean,County,State,Region,fips_id,original_geography,original_geography_type,fips
0,Counties,New Haven,>=18 Years,Non-Medical Setting,45.5,<NA>,43.9,47.2,2018-01-01,18,9009,New Haven County,Connecticut,1,1,New Haven,Counties,9009
1,Counties,New Haven,>=18 Years,Non-Medical Setting,53.0,<NA>,46.0,60.9,2021-01-01,21,9009,New Haven County,Connecticut,1,1,New Haven,Counties,9009
2,Counties,New Haven,Age,>=18 Years,52.4,<NA>,50.6,54.3,2020-01-01,20,9009,New Haven County,Connecticut,1,1,New Haven,Counties,9009
3,Counties,New Haven,Age,>=18 Years,50.2,<NA>,45.4,55.8,2021-01-01,21,9009,New Haven County,Connecticut,1,1,New Haven,Counties,9009
4,Counties,New Haven,Age,>=18 Years,34.0,<NA>,32.6,35.5,2018-01-01,18,9009,New Haven County,Connecticut,1,1,New Haven,Counties,9009


In [46]:
df_counties[df_counties['fips_id'].isna()]

,Geography Type,Geography,Dimension Type,Dimension,Estimate (%),Sample Size,ci_lower,ci_upper,start_cohort_datetime,vaccine_id,FIPS_clean,County,State,Region,fips_id,original_geography,original_geography_type,fips


In [47]:
df_counties[df_counties['Geography'] == 'Valdez-Cordova Census Area']

,Geography Type,Geography,Dimension Type,Dimension,Estimate (%),Sample Size,ci_lower,ci_upper,start_cohort_datetime,vaccine_id,FIPS_clean,County,State,Region,fips_id,original_geography,original_geography_type,fips
20627,Counties,Valdez-Cordova Census Area,Age,>=18 Years,31.0,<NA>,29.4,32.8,2018-01-01,18,2261,Valdez-Cordova Census Area,Alaska,10,12582,Valdez-Cordova Census Area,Counties,2261
20628,Counties,Valdez-Cordova Census Area,Age,>=18 Years,36.8,<NA>,34.9,38.8,2019-01-01,19,2261,Valdez-Cordova Census Area,Alaska,10,12582,Valdez-Cordova Census Area,Counties,2261
20629,Counties,Valdez-Cordova Census Area,>=18 Years,Non-Medical Setting,40.3,<NA>,38.2,42.6,2018-01-01,18,2261,Valdez-Cordova Census Area,Alaska,10,12582,Valdez-Cordova Census Area,Counties,2261


In [49]:
df_counties.drop(columns=['County', 'State', 'Region', 'Geography Type', 'Geography', 'FIPS_clean', 'fips', 'original_geography', 'original_geography_type'], inplace=True)
df_counties.head()

,Dimension Type,Dimension,Estimate (%),Sample Size,ci_lower,ci_upper,start_cohort_datetime,vaccine_id,fips_id
0,>=18 Years,Non-Medical Setting,45.5,<NA>,43.9,47.2,2018-01-01,18,1
1,>=18 Years,Non-Medical Setting,53.0,<NA>,46.0,60.9,2021-01-01,21,1
2,Age,>=18 Years,52.4,<NA>,50.6,54.3,2020-01-01,20,1
3,Age,>=18 Years,50.2,<NA>,45.4,55.8,2021-01-01,21,1
4,Age,>=18 Years,34.0,<NA>,32.6,35.5,2018-01-01,18,1


In [52]:
df_counties.drop(columns=['Sample Size'], inplace=True)

In [53]:
df_counties.head()

,Dimension Type,Dimension,Estimate (%),ci_lower,ci_upper,start_cohort_datetime,vaccine_id,fips_id
0,>=18 Years,Non-Medical Setting,45.5,43.9,47.2,2018-01-01,18,1
1,>=18 Years,Non-Medical Setting,53.0,46.0,60.9,2021-01-01,21,1
2,Age,>=18 Years,52.4,50.6,54.3,2020-01-01,20,1
3,Age,>=18 Years,50.2,45.4,55.8,2021-01-01,21,1
4,Age,>=18 Years,34.0,32.6,35.5,2018-01-01,18,1


In [54]:
df_counties.shape[0]

21832

In [55]:
df_age = df_counties[df_counties['Dimension Type'] == 'Age']

In [56]:
df_age.rename(columns={'Dimension': 'Age'}, inplace=True)
df_age.drop(columns=['Dimension Type'], inplace=True)
print(df_age.shape[0])
df_age.head()

15623


/var/folders/jq/q12_1s5j0kx30cqlm76w1m740000gn/T/ipykernel_12858/3920806466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age.rename(columns={'Dimension': 'Age'}, inplace=True)
/var/folders/jq/q12_1s5j0kx30cqlm76w1m740000gn/T/ipykernel_12858/3920806466.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age.drop(columns=['Dimension Type'], inplace=True)


,Age,Estimate (%),ci_lower,ci_upper,start_cohort_datetime,vaccine_id,fips_id
2,>=18 Years,52.4,50.6,54.3,2020-01-01,20,1
3,>=18 Years,50.2,45.4,55.8,2021-01-01,21,1
4,>=18 Years,34.0,32.6,35.5,2018-01-01,18,1
5,>=18 Years,49.5,47.8,51.3,2019-01-01,19,1
8,>=18 Years,27.2,25.7,29.0,2018-01-01,18,2


In [57]:
df_location = df_counties[df_counties['Dimension Type'] != 'Age']
df_location.head()

,Dimension Type,Dimension,Estimate (%),ci_lower,ci_upper,start_cohort_datetime,vaccine_id,fips_id
0,>=18 Years,Non-Medical Setting,45.5,43.9,47.2,2018-01-01,18,1
1,>=18 Years,Non-Medical Setting,53.0,46.0,60.9,2021-01-01,21,1
6,>=18 Years,Non-Medical Setting,41.8,39.0,44.7,2018-01-01,18,2
7,>=18 Years,Non-Medical Setting,41.1,32.7,49.7,2021-01-01,21,2
13,>=18 Years,Non-Medical Setting,44.0,32.5,55.7,2021-01-01,21,3149


In [58]:
cursor.execute("SELECT vaccine_location, vaccine_location_id FROM vaccine_location_dim")
mapping_data = cursor.fetchall()

vac_location = pd.DataFrame(mapping_data, columns=["vaccine_location", "vaccine_location_id"])
vac_location

,vaccine_location,vaccine_location_id
0,Non-Medical Setting,1
1,School,2
2,Pharmacy/Store,3
3,Workplace,4
4,Medical Setting,5


In [59]:
# we merge df_children with the vaccine_dim
df_location = pd.merge(df_location, vac_location, left_on = ['Dimension'], right_on=['vaccine_location'], how='left')
df_location.head()

,Dimension Type,Dimension,Estimate (%),ci_lower,ci_upper,start_cohort_datetime,vaccine_id,fips_id,vaccine_location,vaccine_location_id
0,>=18 Years,Non-Medical Setting,45.5,43.9,47.2,2018-01-01,18,1,Non-Medical Setting,1
1,>=18 Years,Non-Medical Setting,53.0,46.0,60.9,2021-01-01,21,1,Non-Medical Setting,1
2,>=18 Years,Non-Medical Setting,41.8,39.0,44.7,2018-01-01,18,2,Non-Medical Setting,1
3,>=18 Years,Non-Medical Setting,41.1,32.7,49.7,2021-01-01,21,2,Non-Medical Setting,1
4,>=18 Years,Non-Medical Setting,44.0,32.5,55.7,2021-01-01,21,3149,Non-Medical Setting,1


In [60]:
df_location.drop(columns=['vaccine_location'], inplace=True)
df_location.head()

,Dimension Type,Dimension,Estimate (%),ci_lower,ci_upper,start_cohort_datetime,vaccine_id,fips_id,vaccine_location_id
0,>=18 Years,Non-Medical Setting,45.5,43.9,47.2,2018-01-01,18,1,1
1,>=18 Years,Non-Medical Setting,53.0,46.0,60.9,2021-01-01,21,1,1
2,>=18 Years,Non-Medical Setting,41.8,39.0,44.7,2018-01-01,18,2,1
3,>=18 Years,Non-Medical Setting,41.1,32.7,49.7,2021-01-01,21,2,1
4,>=18 Years,Non-Medical Setting,44.0,32.5,55.7,2021-01-01,21,3149,1


In [61]:
df_location.head()

,Dimension Type,Dimension,Estimate (%),ci_lower,ci_upper,start_cohort_datetime,vaccine_id,fips_id,vaccine_location_id
0,>=18 Years,Non-Medical Setting,45.5,43.9,47.2,2018-01-01,18,1,1
1,>=18 Years,Non-Medical Setting,53.0,46.0,60.9,2021-01-01,21,1,1
2,>=18 Years,Non-Medical Setting,41.8,39.0,44.7,2018-01-01,18,2,1
3,>=18 Years,Non-Medical Setting,41.1,32.7,49.7,2021-01-01,21,2,1
4,>=18 Years,Non-Medical Setting,44.0,32.5,55.7,2021-01-01,21,3149,1


In [62]:
df_age.head()

,Age,Estimate (%),ci_lower,ci_upper,start_cohort_datetime,vaccine_id,fips_id
2,>=18 Years,52.4,50.6,54.3,2020-01-01,20,1
3,>=18 Years,50.2,45.4,55.8,2021-01-01,21,1
4,>=18 Years,34.0,32.6,35.5,2018-01-01,18,1
5,>=18 Years,49.5,47.8,51.3,2019-01-01,19,1
8,>=18 Years,27.2,25.7,29.0,2018-01-01,18,2


In [64]:
# loading cumulative fact table
try:
    insert_query='''
    INSERT INTO influenza_transaction_fact(
        age,
        influenza_transaction_estimate_pct,
        influenza_transaction_ci_lower,
        influenza_transaction_ci_upper,
        influenza_transaction_survey_datetime,
        influenza_transaction_vaccine_id,
        influenza_transaction_fips_id
    )
    VALUES(%s, %s, %s, %s, %s, %s, %s)
    '''
    for index, row in df_age.iterrows():
        start_cohort_date = pd.to_datetime(row['start_cohort_datetime']).date() if pd.notnull(row['start_cohort_datetime']) else None
        vaccine_id = int(row['vaccine_id']) if pd.notnull(row['vaccine_id']) else None
        fips_id = int(row['fips_id']) if pd.notnull(row['fips_id']) else None

        cursor.execute(insert_query,(
        row['Age'],
        row['Estimate (%)'],
        row['ci_lower'],
        row['ci_upper'],
        start_cohort_date,
        vaccine_id,
        fips_id
        ))
    conn.commit()
    print(f"{len(df_age)} records inserted into influenza_transaction_fact.")

except Exception as e:
    print(f"Error occurred: {e}")
    traceback.print_exc(file=sys.stdout)
    conn.rollback()


15623 records inserted into influenza_transaction_fact.


In [67]:
# loading cumulative fact table
try:
    insert_query='''
    INSERT INTO influenza_transaction_fact(
        age,
        influenza_transaction_estimate_pct,
        influenza_transaction_ci_lower,
        influenza_transaction_ci_upper,
        influenza_transaction_survey_datetime,
        influenza_transaction_vaccine_id,
        influenza_transaction_fips_id,
        influenza_transaction_location_id
    )
    VALUES(%s, %s, %s, %s, %s, %s, %s, %s)
    '''
    for index, row in df_location.iterrows():
        start_cohort_date = pd.to_datetime(row['start_cohort_datetime']).date() if pd.notnull(row['start_cohort_datetime']) else None
        vaccine_id = int(row['vaccine_id']) if pd.notnull(row['vaccine_id']) else None
        fips_id = int(row['fips_id']) if pd.notnull(row['fips_id']) else None
        vaccine_location_id = int(row['vaccine_location_id']) if pd.notnull(row['vaccine_location_id']) else None

        cursor.execute(insert_query,(
        row['Dimension Type'],
        row['Estimate (%)'],
        row['ci_lower'],
        row['ci_upper'],
        start_cohort_date,
        vaccine_id,
        fips_id,
        vaccine_location_id
        ))
    conn.commit()
    print(f"{len(df_age)} records inserted into influenza_transaction_fact.")

except Exception as e:
    print(f"Error occurred: {e}")
    traceback.print_exc(file=sys.stdout)
    conn.rollback()


15623 records inserted into influenza_transaction_fact.
